Sistema de recomendación:

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación de películas. El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras con las palabras más frecuentes en los títulos de las películas. Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse:

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer


In [2]:
df_db = pd.read_csv('data.csv', delimiter=',', header='infer')
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 45345 non-null  object 
 1   id_collection         4485 non-null   float64
 2   name_collection       4485 non-null   object 
 3   budget                45345 non-null  int64  
 4   genres                45345 non-null  object 
 5   id                    45345 non-null  int64  
 6   original_language     45334 non-null  object 
 7   overview              44399 non-null  object 
 8   popularity            45345 non-null  float64
 9   production_companies  45345 non-null  object 
 10  production_countries  45345 non-null  object 
 11  release_date          45345 non-null  object 
 12  revenue               45345 non-null  float64
 13  runtime               45099 non-null  float64
 14  spoken_languages      45345 non-null  object 
 15  status             

In [3]:
df_db['title'] = df_db['title'].str.lower()
df_db = df_db.dropna(subset=['title'])
corpus =df_db['title'].tolist()
print(corpus)

['toy story', 'jumanji', 'grumpier old men', 'waiting to exhale', 'father of the bride part ii', 'heat', 'sabrina', 'tom and huck', 'sudden death', 'goldeneye', 'the american president', 'dracula: dead and loving it', 'balto', 'nixon', 'cutthroat island', 'casino', 'sense and sensibility', 'four rooms', 'ace ventura: when nature calls', 'money train', 'get shorty', 'copycat', 'assassins', 'powder', 'leaving las vegas', 'othello', 'now and then', 'persuasion', 'the city of lost children', 'shanghai triad', 'dangerous minds', 'twelve monkeys', 'wings of courage', 'babe', 'carrington', 'dead man walking', 'across the sea of time', 'it takes two', 'clueless', 'cry, the beloved country', 'richard iii', 'dead presidents', 'restoration', 'mortal kombat', 'to die for', 'how to make an american quilt', 'se7en', 'pocahontas', 'when night is falling', 'the usual suspects', 'guardian angel', 'mighty aphrodite', 'lamerica', 'the big green', 'georgia', 'kids of the round table', 'home for the holida

In [4]:
porter_stemmer = PorterStemmer()

def stemming_tokenizer(str_input):
    # Eliminar signos de puntuación

    str_input=re.sub(r"[^\w\s]", "", str_input)
    # Eliminar números    
    str_input = re.sub(r"\d+", "", str_input)
    
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [word for word in words if len(word) > 1]
    words = [word.strip() for word in words]
    words = [porter_stemmer.stem(word) for word in words]

    return words
tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

#l2_df = pd.DataFrame(X.toarray(), columns=l2_vectorizer.get_feature_names_out())
#l2_df
print(tfidf_matrix.toarray())


/home/juanjo/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/juanjo/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaf

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
query = "toy story 3"
query_vector = tfidf_vectorizer.transform([query])
print(query_vector.toarray())
similarities = cosine_similarity(query_vector, tfidf_matrix)

[[0. 0. 0. ... 0. 0. 0.]]


In [6]:
#Primero vemos si la pelicula no está, si está no se considera en el indice de busqueda y si no igual se busca los mas parecidos
try:
    toy_index = corpus.index(query)
    top_indices = similarities.argsort()[0][::-1][:6]
    top_indices=top_indices[1:6]
except:
    top_indices = similarities.argsort()[0][::-1][:5]

print(top_indices)
for index in top_indices:
    print(corpus[index])

[ 2994 15335 16139  4794  2139]
toy story 2
toy story 3
the toy
the toy
toys


In [7]:
#guardo la matriz tfidf que se usará en el API

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf.to_csv("tfidf_matrix.csv", index=False)

KeyboardInterrupt: 

In [ ]:
# Cargamos la matriz TF-IDF desde el archivo CSV
df_tfidf = pd.read_csv("tfidf_matrix.csv")

# Obtenemos el vector TF-IDF de la consulta
query = "Toy"
query_vector = tfidf_vectorizer.transform([query])

# Calculamos las similitudes de coseno entre el vector de consulta y la matriz TF-IDF
similarities = cosine_similarity(query_vector, df_tfidf)

# Obtener los índices de las películas más similares
try:
    toy_index = corpus.index(query)
    top_indices = similarities.argsort()[0][::-1][:6]
    top_indices=top_indices[1:6]
except:
    top_indices = similarities.argsort()[0][::-1][:5]

print(top_indices)
for index in top_indices:
    print(corpus[index])